In [1]:
cd matlab_ai/Mask_RCNN

C:\Users\Home\Desktop\matlab_ai\Mask_RCNN


## 사용하는 코드) 최종

In [ ]:
pip install pyttsx3
pip install playsound
pip install pygame

In [9]:
#찐최종
import os
import numpy as np
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib
import cv2
import pygame
import threading
import time
from queue import Queue

class_names = [
    'bicycle', 'bus', 'car', 'carrier', 'cat', 'dog', 'motorcycle', 
    'movable_signage', 'person', 'scooter', 'stroller', 'truck', 
    'wheelchair', 'barricade', 'bench', 'bollard', 'chair', 'fire_hydrant', 
    'kiosk', 'parking_meter', 'pole', 'potted_plant', 'power_controller', 
    'stop', 'table', 'traffic_light', 'traffic_light_controller', 
    'traffic_sign', 'tree_trunk', 'caution_zone', 'sidewalk', 
    'roadway', 'braille_guide_blocks'
]

# Category mapping
category_mapping = {
    'bicycle': 'bicycle',
    'bus': 'bus',
    'car': 'vehicle',
    'carrier': 'moving_obstacle',
    'cat': 'moving_obstacle',
    'dog': 'moving_obstacle',
    'motorcycle': 'motorcycle',
    'movable_signage': 'moving_obstacle',
    'person': 'person',
    'scooter': 'motorcycle',
    'stroller': 'moving_obstacle',
    'truck': 'vehicle',
    'wheelchair': 'moving_obstacle',
    'barricade': 'fixed_obstacle',
    'bench': 'fixed_obstacle',
    'bollard': 'fixed_obstacle',
    'chair': 'fixed_obstacle',
    'fire_hydrant': 'fixed_obstacle',
    'kiosk': 'fixed_obstacle',
    'parking_meter': 'fixed_obstacle',
    'pole': 'pole',
    'potted_plant': 'fixed_obstacle',
    'power_controller': 'fixed_obstacle',
    'stop': 'fixed_obstacle',
    'table': 'fixed_obstacle',
    'traffic_light': 'fixed_obstacle',
    'traffic_light_controller': 'fixed_obstacle',
    'traffic_sign': 'fixed_obstacle',
    'tree_trunk': 'tree_trunk',
    'caution_zone': 'caution_zone',
    'sidewalk': 'sidewalk',
    'roadway': 'roadway',
    'braille_guide_blocks': 'braille_guide_blocks'
}

# Audio file paths
audio_files = {
    'bicycle': 'C:/Users/Home/Desktop/matlab_ai/audio/자전거.mp3',
    'bus': 'C:/Users/Home/Desktop/matlab_ai/audio/버스.mp3',
    'vehicle': 'C:/Users/Home/Desktop/matlab_ai/audio/차량.mp3',
    'moving_obstacle': 'C:/Users/Home/Desktop/matlab_ai/audio/이동장애물.mp3',
    'motorcycle': 'C:/Users/Home/Desktop/matlab_ai/audio/오토바이.mp3',
    'fixed_obstacle': 'C:/Users/Home/Desktop/matlab_ai/audio/고정장애물.mp3',
    'pole': 'C:/Users/Home/Desktop/matlab_ai/audio/기둥.mp3',
    'tree_trunk': 'C:/Users/Home/Desktop/matlab_ai/audio/나무기둥.mp3',
    'caution_zone': 'C:/Users/Home/Desktop/matlab_ai/audio/주의구역.mp3',
    'sidewalk': 'C:/Users/Home/Desktop/matlab_ai/audio/인도.mp3',
    'roadway': 'C:/Users/Home/Desktop/matlab_ai/audio/차도.mp3',
    'braille_guide_blocks': 'C:/Users/Home/Desktop/matlab_ai/audio/점자블록.mp3',
    'person': 'C:/Users/Home/Desktop/matlab_ai/audio/사람.mp3',
    '전방에는': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는.mp3',
    '좌측에는': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는.mp3',
    '우측에는': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는.mp3',
    '전방에는 점자블록': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는 점자블록.mp3',
    '좌측에는 점자블록': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는 점자블록.mp3',
    '우측에는 점자블록': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는 점자블록.mp3',
    '전방에는 주의구역': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는 주의구역.mp3',
    '좌측에는 주의구역': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는 주의구역.mp3',
    '우측에는 주의구역': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는 주의구역.mp3',
}

# Configuration class for Mask R-CNN
class InferenceConfig(Config):
    NAME = "coco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 33  # Number of classes including background
    DETECTION_MIN_CONFIDENCE = 0.7  # General detection threshold

config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", model_dir="Mask_RCNN", config=config)
model.load_weights(r"C:\Users\Home\Desktop\tm_mask_rcnn_test_0200.h5", by_name=True)

def calculate_area_and_center(mask):
    area = np.sum(mask)
    indices = np.where(mask)
    if len(indices[0]) == 0 or len(indices[1]) == 0:
        return 0, (0, 0)
    y_center = np.mean(indices[0])
    x_center = np.mean(indices[1])
    return area, (int(x_center), int(y_center))

def play_audio(file_path):
    if os.path.exists(file_path):
        pygame.mixer.init()
        pygame.mixer.music.load(file_path)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
    else:
        print(f"Warning: Audio file {file_path} does not exist.")

def audio_player(audio_queue, interrupt_event):
    while True:
        audio_file = audio_queue.get()
        if not interrupt_event.is_set():
            play_audio(audio_file)
        else:
            while not audio_queue.empty():
                audio_queue.get_nowait()
            play_audio(audio_file)  # Play the interrupt message
            interrupt_event.clear()
        audio_queue.task_done()

# GPU settings
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Video capture
cap = cv2.VideoCapture(1)

# Desired size for the model input
model_input_size = (800, 600)  # Adjusted size for better performance

# Output screen size
output_screen_size = (640, 480)  # Desired output screen size

# Specific threshold for certain classes
specific_thresholds = {
    'sidewalk': 55.0,
    'roadway': 55.0,
    'caution_zone': 30.0,
    'braille_guide_blocks': 30.0
}

# Audio queue and interrupt event
audio_queue = Queue()
interrupt_event = threading.Event()
audio_thread = threading.Thread(target=audio_player, args=(audio_queue, interrupt_event), daemon=True)
audio_thread.start()

# Audio output control
audio_duration = 7  # seconds
last_audio_time = time.time()

def is_within_box(center, box_top_left, box_bottom_right):
    x, y = center
    return (box_top_left[0] <= x <= box_bottom_right[0]) and (box_top_left[1] <= y <= box_bottom_right[1])

def get_section(center, box_top_left, section_width):
    x = center[0]
    if x < box_top_left[0] + section_width:
        return 'left'
    elif x < box_top_left[0] + 2 * section_width:
        return 'center'
    else:
        return 'right'

def check_and_play_audio(audio_key):
    global last_audio_time
    current_time = time.time()
    if current_time - last_audio_time >= audio_duration:
        audio_file = audio_files.get(audio_key)
        if audio_file:
            audio_queue.put(audio_file)
            last_audio_time = current_time

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection on the resized frame
    resized_frame = cv2.resize(frame, model_input_size)
    results = model.detect([resized_frame], verbose=0)
    r = results[0]

    detected_classes = {'center': set(), 'left': set(), 'right': set()}
    
    # Scale up bounding boxes and masks
    for i in range(len(r['class_ids'])):
        mask = r['masks'][:, :, i]
        area, center = calculate_area_and_center(mask)
        (x, y, w, h) = cv2.boundingRect(mask.astype(np.uint8))
        class_id = r['class_ids'][i]
        class_name = class_names[class_id - 1]
        score = r['scores'][i]
        score_percentage = int(score * 100)

        # Determine the threshold based on class
        threshold = specific_thresholds.get(category_mapping.get(class_name, class_name), config.DETECTION_MIN_CONFIDENCE * 100)

        # Ensure no NaN values are used
        if np.isnan(x) or np.isnan(y) or np.isnan(w) or np.isnan(h):
            continue

        # Determine if the object should be displayed based on its class and score
        if score_percentage >= threshold:
            # Scale up the coordinates to match the original frame size
            scale_x = frame.shape[1] / model_input_size[0]
            scale_y = frame.shape[0] / model_input_size[1]
            x, y, w, h = int(x * scale_x), int(y * scale_y), int(w * scale_x), int(h * scale_y)
            center = (int(center[0] * scale_x), int(center[1] * scale_y))

            # Draw the bounding box and mask on the original frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            cv2.putText(frame, f'{class_name}: {score_percentage}%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw the detection box and its sections
            box_width, box_height = 600, 250
            vertical_offset = 50
            top_left = ((frame.shape[1] - box_width) // 2, (frame.shape[0] - box_height) // 2 + vertical_offset)
            bottom_right = (top_left[0] + box_width, top_left[1] + box_height)
            section_width = box_width // 3
            cv2.rectangle(frame, top_left, bottom_right, (255, 0, 0), 2)
            for i in range(3):
                section_start = top_left[0] + i * section_width
                section_end = section_start + section_width
                cv2.line(frame, (section_start, top_left[1]), (section_start, bottom_right[1]), (0, 255, 255), 2)
                cv2.line(frame, (section_end, top_left[1]), (section_end, bottom_right[1]), (0, 255, 255), 2)

            # Check if the object's center is within the blue box
            if is_within_box(center, top_left, bottom_right):
                section = get_section(center, top_left, section_width)
                detected_classes[section].add(class_name)

    # Check for interrupt conditions
    interrupt_detected = False
    interrupt_message = None
    for section in ['left', 'center', 'right']:
        if 'caution_zone' in detected_classes[section]:
            if section == 'left':
                interrupt_message = '좌측에는 주의구역'
            elif section == 'center':
                interrupt_message = '전방에는 주의구역'
            elif section == 'right':
                interrupt_message = '우측에는 주의구역'
            interrupt_detected = True
            break
        elif 'braille_guide_blocks' in detected_classes[section]:
            if section == 'left':
                interrupt_message = '좌측에는 점자블록'
            elif section == 'center':
                interrupt_message = '전방에는 점자블록'
            elif section == 'right':
                interrupt_message = '우측에는 점자블록'
            interrupt_detected = True
            break

    if interrupt_detected:
        interrupt_event.set()
        audio_queue.put(audio_files.get(interrupt_message))
        last_audio_time = time.time()
    else:
        current_time = time.time()
        if current_time - last_audio_time >= audio_duration:
            if detected_classes['center']:
                audio_queue.put(audio_files['전방에는'])
                for class_name in detected_classes['center']:
                    audio_queue.put(audio_files.get(category_mapping.get(class_name, class_name)))
            if detected_classes['left']:
                audio_queue.put(audio_files['좌측에는'])
                for class_name in detected_classes['left']:
                    audio_queue.put(audio_files.get(category_mapping.get(class_name, class_name)))
            if detected_classes['right']:
                audio_queue.put(audio_files['우측에는'])
                for class_name in detected_classes['right']:
                    audio_queue.put(audio_files.get(category_mapping.get(class_name, class_name)))
            last_audio_time = current_time

    # Resize the frame to the output screen size
    output_frame = cv2.resize(frame, output_screen_size)


    # Show the resized frame
    cv2.imshow('Mask R-CNN Detection', output_frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


In [11]:
#일단이건 킵 ( 나중에 사용 가능성 있음 )
import os
import numpy as np
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib
import cv2
import pygame
import threading
import time
from queue import Queue

class_names = [
    'bicycle', 'bus', 'car', 'carrier', 'cat', 'dog', 'motorcycle', 
    'movable_signage', 'person', 'scooter', 'stroller', 'truck', 
    'wheelchair', 'barricade', 'bench', 'bollard', 'chair', 'fire_hydrant', 
    'kiosk', 'parking_meter', 'pole', 'potted_plant', 'power_controller', 
    'stop', 'table', 'traffic_light', 'traffic_light_controller', 
    'traffic_sign', 'tree_trunk', 'caution_zone', 'sidewalk', 
    'roadway', 'braille_guide_blocks'
]

# Category mapping
category_mapping = {
    'bicycle': 'bicycle',
    'bus': 'bus',
    'car': 'vehicle',
    'carrier': 'moving_obstacle',
    'cat': 'moving_obstacle',
    'dog': 'moving_obstacle',
    'motorcycle': 'motorcycle',
    'movable_signage': 'moving_obstacle',
    'person': 'person',
    'scooter': 'motorcycle',
    'stroller': 'moving_obstacle',
    'truck': 'vehicle',
    'wheelchair': 'moving_obstacle',
    'barricade': 'fixed_obstacle',
    'bench': 'fixed_obstacle',
    'bollard': 'fixed_obstacle',
    'chair': 'fixed_obstacle',
    'fire_hydrant': 'fixed_obstacle',
    'kiosk': 'fixed_obstacle',
    'parking_meter': 'fixed_obstacle',
    'pole': 'pole',
    'potted_plant': 'fixed_obstacle',
    'power_controller': 'fixed_obstacle',
    'stop': 'fixed_obstacle',
    'table': 'fixed_obstacle',
    'traffic_light': 'fixed_obstacle',
    'traffic_light_controller': 'fixed_obstacle',
    'traffic_sign': 'fixed_obstacle',
    'tree_trunk': 'tree_trunk',
    'caution_zone': 'caution_zone',
    'sidewalk': 'sidewalk',
    'roadway': 'roadway',
    'braille_guide_blocks': 'braille_guide_blocks'
}

# Audio file paths
audio_files = {
    'bicycle': 'C:/Users/Home/Desktop/matlab_ai/audio/자전거.mp3',
    'bus': 'C:/Users/Home/Desktop/matlab_ai/audio/버스.mp3',
    'vehicle': 'C:/Users/Home/Desktop/matlab_ai/audio/차량.mp3',
    'moving_obstacle': 'C:/Users/Home/Desktop/matlab_ai/audio/이동장애물.mp3',
    'motorcycle': 'C:/Users/Home/Desktop/matlab_ai/audio/오토바이.mp3',
    'fixed_obstacle': 'C:/Users/Home/Desktop/matlab_ai/audio/고정장애물.mp3',
    'pole': 'C:/Users/Home/Desktop/matlab_ai/audio/기둥.mp3',
    'tree_trunk': 'C:/Users/Home/Desktop/matlab_ai/audio/나무기둥.mp3',
    'caution_zone': 'C:/Users/Home/Desktop/matlab_ai/audio/주의구역.mp3',
    'sidewalk': 'C:/Users/Home/Desktop/matlab_ai/audio/인도.mp3',
    'roadway': 'C:/Users/Home/Desktop/matlab_ai/audio/차도.mp3',
    'braille_guide_blocks': 'C:/Users/Home/Desktop/matlab_ai/audio/점자블록.mp3',
    'person': 'C:/Users/Home/Desktop/matlab_ai/audio/사람.mp3',
    '전방에는': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는.mp3',
    '좌측에는': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는.mp3',
    '우측에는': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는.mp3'
}

# Configuration class for Mask R-CNN
class InferenceConfig(Config):
    NAME = "coco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 33  # Number of classes including background
    DETECTION_MIN_CONFIDENCE = 0.65  # General detection threshold

config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", model_dir="Mask_RCNN", config=config)
model.load_weights(r"C:\Users\Home\Desktop\tm_mask_rcnn_test_0100.h5", by_name=True)

def calculate_area_and_center(mask):
    area = np.sum(mask)
    indices = np.where(mask)
    if len(indices[0]) == 0 or len(indices[1]) == 0:
        return 0, (0, 0)
    y_center = np.mean(indices[0])
    x_center = np.mean(indices[1])
    return area, (int(x_center), int(y_center))

def play_audio(file_path):
    if os.path.exists(file_path):
        pygame.mixer.init()
        pygame.mixer.music.load(file_path)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
    else:
        print(f"Warning: Audio file {file_path} does not exist.")

def audio_player(audio_queue):
    while True:
        audio_file = audio_queue.get()
        play_audio(audio_file)
        audio_queue.task_done()

# GPU settings
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Video capture
cap = cv2.VideoCapture(1)

# Desired size for the model input
model_input_size = (800, 600)  # Adjusted size for better performance

# Output screen size
output_screen_size = (640, 480)  # Desired output screen size

# Specific threshold for certain classes
specific_thresholds = {
    'sidewalk': 80.0,
    'roadway': 80.0,
    'caution_zone': 20.0,
    'braille_guide_blocks': 20.0
}

# Audio queue
audio_queue = Queue()
audio_thread = threading.Thread(target=audio_player, args=(audio_queue,), daemon=True)
audio_thread.start()

# Previous detected class to avoid repetitive audio
prev_detected_class = None

# Audio output control
audio_lock = threading.Lock()
audio_duration = 8  # seconds
last_audio_time = time.time()

def is_within_box(center, box_top_left, box_bottom_right):
    x, y = center
    return (box_top_left[0] <= x <= box_bottom_right[0]) and (box_top_left[1] <= y <= box_bottom_right[1])

def get_section(center, box_top_left, section_width):
    x = center[0]
    if x < box_top_left[0] + section_width:
        return 'left'
    elif x < box_top_left[0] + 2 * section_width:
        return 'center'
    else:
        return 'right'

def check_and_play_audio(audio_key):
    global last_audio_time
    current_time = time.time()
    if current_time - last_audio_time >= audio_duration:
        audio_file = audio_files.get(audio_key)
        if audio_file:
            audio_queue.put(audio_file)
            last_audio_time = current_time

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection on the resized frame
    resized_frame = cv2.resize(frame, model_input_size)
    results = model.detect([resized_frame], verbose=0)
    r = results[0]

    detected_classes = {'center': set(), 'left': set(), 'right': set()}
    
    # Scale up bounding boxes and masks
    for i in range(len(r['class_ids'])):
        mask = r['masks'][:, :, i]
        area, center = calculate_area_and_center(mask)
        (x, y, w, h) = cv2.boundingRect(mask.astype(np.uint8))
        class_id = r['class_ids'][i]
        class_name = class_names[class_id - 1]
        score = r['scores'][i]
        score_percentage = int(score * 100)

        # Determine the threshold based on class
        threshold = specific_thresholds.get(category_mapping.get(class_name, class_name), config.DETECTION_MIN_CONFIDENCE * 100)

        # Ensure no NaN values are used
        if np.isnan(x) or np.isnan(y) or np.isnan(w) or np.isnan(h):
            continue

        # Determine if the object should be displayed based on its class and score
        if score_percentage >= threshold:
            # Scale up the coordinates to match the original frame size
            scale_x = frame.shape[1] / model_input_size[0]
            scale_y = frame.shape[0] / model_input_size[1]
            x, y, w, h = int(x * scale_x), int(y * scale_y), int(w * scale_x), int(h * scale_y)
            center = (int(center[0] * scale_x), int(center[1] * scale_y))

            # Draw the bounding box and mask on the original frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            cv2.putText(frame, f'{class_name}: {score_percentage}%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw the detection box and its sections
            box_width, box_height = 600, 250
            vertical_offset = 50
            top_left = ((frame.shape[1] - box_width) // 2, (frame.shape[0] - box_height) // 2 + vertical_offset)
            bottom_right = (top_left[0] + box_width, top_left[1] + box_height)
            section_width = box_width // 3
            cv2.rectangle(frame, top_left, bottom_right, (255, 0, 0), 2)
            for i in range(3):
                section_start = top_left[0] + i * section_width
                section_end = section_start + section_width
                cv2.line(frame, (section_start, top_left[1]), (section_start, bottom_right[1]), (0, 255, 255), 2)
                cv2.line(frame, (section_end, top_left[1]), (section_end, bottom_right[1]), (0, 255, 255), 2)

            # Check if the object's center is within the blue box
            if is_within_box(center, top_left, bottom_right):
                section = get_section(center, top_left, section_width)
                detected_classes[section].add(class_name)

    current_time = time.time()
    if current_time - last_audio_time >= audio_duration:
        if detected_classes['center']:
            audio_queue.put(audio_files['전방에는'])
            for class_name in detected_classes['center']:
                audio_queue.put(audio_files.get(category_mapping.get(class_name, class_name)))
        if detected_classes['left']:
            audio_queue.put(audio_files['좌측에는'])
            for class_name in detected_classes['left']:
                audio_queue.put(audio_files.get(category_mapping.get(class_name, class_name)))
        if detected_classes['right']:
            audio_queue.put(audio_files['우측에는'])
            for class_name in detected_classes['right']:
                audio_queue.put(audio_files.get(category_mapping.get(class_name, class_name)))
        last_audio_time = current_time

    # Resize the frame to the output screen size
    output_frame = cv2.resize(frame, output_screen_size)

    # Show the resized frame
    cv2.imshow('Mask R-CNN Detection', output_frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import tkinter as tk
from tkinter import Label
import cv2
from PIL import Image, ImageTk
import threading
import time
import numpy as np
import pygame
import os
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib

class_names = [
    'bicycle', 'bus', 'car', 'carrier', 'cat', 'dog', 'motorcycle', 
    'movable_signage', 'person', 'scooter', 'stroller', 'truck', 
    'wheelchair', 'barricade', 'bench', 'bollard', 'chair', 'fire_hydrant', 
    'kiosk', 'parking_meter', 'pole', 'potted_plant', 'power_controller', 
    'stop', 'table', 'traffic_light', 'traffic_light_controller', 
    'traffic_sign', 'tree_trunk', 'caution_zone', 'sidewalk', 
    'roadway', 'braille_guide_blocks'
]

audio_files = {
    'bicycle': 'C:/Users/Home/Desktop/matlab_ai/audio/자전거.mp3',
    'bus': 'C:/Users/Home/Desktop/matlab_ai/audio/버스.mp3',
    'vehicle': 'C:/Users/Home/Desktop/matlab_ai/audio/차량.mp3',
    'moving_obstacle': 'C:/Users/Home/Desktop/matlab_ai/audio/이동장애물.mp3',
    'motorcycle': 'C:/Users/Home/Desktop/matlab_ai/audio/오토바이.mp3',
    'fixed_obstacle': 'C:/Users/Home/Desktop/matlab_ai/audio/고정장애물.mp3',
    'pole': 'C:/Users/Home/Desktop/matlab_ai/audio/기둥.mp3',
    'tree_trunk': 'C:/Users/Home/Desktop/matlab_ai/audio/나무기둥.mp3',
    'caution_zone': 'C:/Users/Home/Desktop/matlab_ai/audio/주의구역.mp3',
    'sidewalk': 'C:/Users/Home/Desktop/matlab_ai/audio/인도.mp3',
    'roadway': 'C:/Users/Home/Desktop/matlab_ai/audio/차도.mp3',
    'braille_guide_blocks': 'C:/Users/Home/Desktop/matlab_ai/audio/점자블록.mp3',
    'person': 'C:/Users/Home/Desktop/matlab_ai/audio/사람.mp3',
    '전방에는': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는.mp3',
    '좌측에는': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는.mp3',
    '우측에는': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는.mp3',
    '전방에는 점자블록': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는 점자블록.mp3',
    '좌측에는 점자블록': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는 점자블록.mp3',
    '우측에는 점자블록': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는 점자블록.mp3',
    '전방에는 주의구역': 'C:/Users/Home/Desktop/matlab_ai/audio/전방에는 주의구역.mp3',
    '좌측에는 주의구역': 'C:/Users/Home/Desktop/matlab_ai/audio/좌측에는 주의구역.mp3',
    '우측에는 주의구역': 'C:/Users/Home/Desktop/matlab_ai/audio/우측에는 주의구역.mp3',
}

# Configuration class for Mask R-CNN
class InferenceConfig(Config):
    NAME = "coco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 33  # Number of classes including background
    DETECTION_MIN_CONFIDENCE = 0.6  # General detection threshold

config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", model_dir="Mask_RCNN", config=config)
model.load_weights(r"C:\Users\Home\Desktop\tm_mask_rcnn_test_0100.h5", by_name=True)

def calculate_area_and_center(mask):
    area = np.sum(mask)
    indices = np.where(mask)
    if len(indices[0]) == 0 or len(indices[1]) == 0:
        return 0, (0, 0)
    y_center = np.mean(indices[0])
    x_center = np.mean(indices[1])
    return area, (int(x_center), int(y_center))

def play_audio(file_path):
    if os.path.exists(file_path):
        pygame.mixer.init()
        pygame.mixer.music.load(file_path)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
    else:
        print(f"Warning: Audio file {file_path} does not exist.")

def audio_player(audio_queue, interrupt_event):
    while True:
        audio_file = audio_queue.get()
        if not interrupt_event.is_set():
            play_audio(audio_file)
        else:
            while not audio_queue.empty():
                audio_queue.get_nowait()
            play_audio(audio_file)  # Play the interrupt message
            interrupt_event.clear()
        audio_queue.task_done()

# GPU settings
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection and Audio Output")

        # Create video display label
        self.video_label = Label(root)
        self.video_label.pack()

        # Create status label
        self.status_label = Label(root, text="Status: Waiting...")
        self.status_label.pack()

        # Initialize video capture
        self.cap = cv2.VideoCapture(1)

        # Initialize audio queue and event
        self.audio_queue = Queue()
        self.interrupt_event = threading.Event()
        self.audio_thread = threading.Thread(target=audio_player, args=(self.audio_queue, self.interrupt_event), daemon=True)
        self.audio_thread.start()

        # Start video update thread
        self.video_thread = threading.Thread(target=self.update_video)
        self.video_thread.daemon = True
        self.video_thread.start()

        self.last_audio_time = time.time()
        self.audio_duration = 7  # seconds

    def update_video(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                continue

            # Run detection on the resized frame
            model_input_size = (800, 600)
            resized_frame = cv2.resize(frame, model_input_size)
            results = model.detect([resized_frame], verbose=0)
            r = results[0]

            detected_classes = {'center': set(), 'left': set(), 'right': set()}

            # Scale up bounding boxes and masks
            for i in range(len(r['class_ids'])):
                mask = r['masks'][:, :, i]
                area, center = calculate_area_and_center(mask)
                (x, y, w, h) = cv2.boundingRect(mask.astype(np.uint8))
                class_id = r['class_ids'][i]
                class_name = class_names[class_id - 1]
                score = r['scores'][i]
                score_percentage = int(score * 100)

                # Determine the threshold based on class
                threshold = specific_thresholds.get(category_mapping.get(class_name, class_name), config.DETECTION_MIN_CONFIDENCE * 100)

                # Ensure no NaN values are used
                if np.isnan(x) or np.isnan(y) or np.isnan(w) or np.isnan(h):
                    continue

                # Determine if the object should be displayed based on its class and score
                if score_percentage >= threshold:
                    # Scale up the coordinates to match the original frame size
                    scale_x = frame.shape[1] / model_input_size[0]
                    scale_y = frame.shape[0] / model_input_size[1]
                    x, y, w, h = int(x * scale_x), int(y * scale_y), int(w * scale_x), int(h * scale_y)
                    center = (int(center[0] * scale_x), int(center[1] * scale_y))

                    # Draw the bounding box and mask on the original frame
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.circle(frame, center, 5, (0, 0, 255), -1)
                    cv2.putText(frame, f'{class_name}: {score_percentage}%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Draw the detection box and its sections
                    box_width, box_height = w, h
                    detection_area = {
                        'center': (x + box_width // 2, y + box_height // 2),
                        'left': (x + box_width // 4, y + box_height // 2),
                        'right': (x + 3 * box_width // 4, y + box_height // 2)
                    }
                    
                    # Determine position relative to the center
                    if detection_area['center'][0] < frame.shape[1] // 3:
                        detected_classes['left'].add(class_name)
                    elif detection_area['center'][0] > 2 * frame.shape[1] // 3:
                        detected_classes['right'].add(class_name)
                    else:
                        detected_classes['center'].add(class_name)

            # Determine if the audio should be played
            current_time = time.time()
            if current_time - self.last_audio_time > self.audio_duration:
                self.last_audio_time = current_time
                self.interrupt_event.set()
                self.audio_queue.put(audio_files.get('전방에는', ''))

                for position in ['center', 'left', 'right']:
                    for class_name in detected_classes[position]:
                        audio_file = audio_files.get(class_name, '')
                        if audio_file:
                            self.audio_queue.put(audio_file)
                            self.interrupt_event.clear()

            # Convert frame to ImageTk format
            image = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=image)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)

            # Update status
            self.status_label.configure(text="Status: Processing...")

            self.root.update_idletasks()
            self.root.update()

    def play_audio(self, file_path):
        if os.path.exists(file_path):
            pygame.mixer.init()
            pygame.mixer.music.load(file_path)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                time.sleep(0.1)
        else:
            print(f"Warning: Audio file {file_path} does not exist.")

# Create and run the application
root = tk.Tk()
app = App(root)
root.mainloop()


## 사용하는 코드) TTS 사용하여 MP3 파일 만들기

In [15]:
from gtts import gTTS
import os

# 음성 파일 저장할 폴더 경로
audio_files_folder = r"C:\Users\Home\Desktop\matlab_ai\audio"

# 음성 파일 생성 함수
def create_audio_file(text, filename):
    tts = gTTS(text=text, lang='ko')  # 한국어로 설정
    tts.save(os.path.join(audio_files_folder, filename))
    print(f'Audio file "{filename}" created successfully.')


Audio file "전방에는 주의구역.mp3" created successfully.
Audio file "좌측에는 주의구역.mp3" created successfully.
Audio file "우측에는 주의구역.mp3" created successfully.
Audio file "전방에는 점자블록.mp3" created successfully.
Audio file "좌측에는 점자블록.mp3" created successfully.
Audio file "우측에는 점자블록.mp3" created successfully.


In [ ]:
# 음성 파일 생성
create_audio_file('전방에는 주의구역', '전방에는 주의구역.mp3')
create_audio_file('좌측에는 주의구역', '좌측에는 주의구역.mp3')
create_audio_file('우측에는 주의구역', '우측에는 주의구역.mp3')
create_audio_file('전방에는 점자블록', '전방에는 점자블록.mp3')
create_audio_file('좌측에는 점자블록', '좌측에는 점자블록.mp3')
create_audio_file('우측에는 점자블록', '우측에는 점자블록.mp3')
create_audio_file('전방에는', '전방에는.mp3')
create_audio_file('좌측에는', '좌측에는.mp3')
create_audio_file('우측에는', '우측에는.mp3')
create_audio_file('자전거', '자전거.mp3')
create_audio_file('버스', '버스.mp3')
create_audio_file('차량', '차량.mp3')
create_audio_file('이동장애물', '이동장애물.mp3')
create_audio_file('오토바이', '오토바이.mp3')
create_audio_file('고정장애물', '고정장애물.mp3')
create_audio_file('기둥', '기둥.mp3')
create_audio_file('나무기둥', '나무기둥.mp3')
create_audio_file('주의구역', '주의구역.mp3')
create_audio_file('인도', '인도.mp3')
create_audio_file('차도', '차도.mp3')
create_audio_file('점자블록', '점자블록.mp3')
create_audio_file('사람', '사람.mp3')

In [12]:
pip install --upgrade google-cloud-texttospeech

Note: you may need to restart the kernel to use updated packages.


In [2]:
def synthesize_text(text):
    # "texttospeech import"
    from google.cloud import texttospeech        

    client = texttospeech.TextToSpeechClient()

    # 최대 길이를 200으로 지정 (지나치게 길어지면 에러 발생)
    max_length = 200    
    # . 단위로 문장 분리
    words = text.split('. ')
    sentences = []
    current_sentence = ''
    for word in words:
        if len(current_sentence + word) <= max_length:
            current_sentence += word + ' '
        else:
            sentences.append(current_sentence.strip() + '.')
            current_sentence = word + ' '
    if current_sentence:
        sentences.append(current_sentence.strip() + '.')

    

    # 빈 배열 생성
    audio_data = []

    # 문장 개수 단위로 텍스트 변환
    for sentence in sentences:
        input_text = texttospeech.SynthesisInput(text=sentence)

        # 오디오 설정 (예제에서는 한국어, 남성C)
        voice = texttospeech.VoiceSelectionParams(
            language_code="ko-KR",
            name="ko-KR-Neural2-C",
            ssml_gender=texttospeech.SsmlVoiceGender.MALE,
        )

        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3
        )

        response = client.synthesize_speech(
            request={"input": input_text, "voice": voice, "audio_config": audio_config}
        )

        audio_data.append(response.audio_content)
  
    audio_data = b"".join(audio_data)
    
    # audio 폴더 안에 output.mp3라는 이름으로 파일 생성
    with open("audio/output.mp3", "wb") as out:        
        out.write(audio_data)
        print('오디오 파일 생성')


synthesize_text("안녕하세요. 팀노바 티스토리 블로그에 오신 것을 환영합니다.")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.